찬우: Output 데이터 양산

In [1]:
import json
import re

# 대체어 그룹 정의
replacement_groups_1 = {   'backpack': ['sack', 'sacks'],
    't shirt': ['blazer', 'jackets'],
    'vest': ['harnesses'],
    'snowsuit': ['jacket', 'coveralls', 'overalls', 'nightgown', 'pajamas', 'rompers', 'suit'],
    'pin button': ['magnet', 'magnets'],
    'acrilic picture': ['embroidery'],
    'bush': ['birch'],
    'moss': ['turf', 'lawn', 'weed', 'weeds', 'grass', 'grasses', 'paste'],
    'japanese macaque': ['volunteer', 'officer', 'officers', 'freethinkers', 'fighter', 'coach', 'leader', 'adolescent', 'juvenile', 'teen', 'young', 'youth', 'cleaner', 'cardinals', 'revelers', 'cast', 'dude', 'friend', 'friends', 'mate', 'mates', 'cameraman', 'cowboy'],
    'chimpanzee': ['politician', 'bicyclist', 'bicyclists', 'cyclist', 'cyclists', 'juggler', 'jockeys', 'pilot', 'pilots', 'zombie', 'zombies', 'seller', 'vender', 'vendor'],
    'homo erectus': ['jogger', 'master', 'conductor', 'conductors'],
    'candle holder': ['goblet'],
    'crape': ['ripples'],
    'ripples': ['crape'],
    'cartoon': ['magazine', 'magazines'],
    'thimble': ['glove', 'gloves', 'mitt', 'mitten', 'mittens', 'mitts'],
    'mahjong': ['checkers'],
    'peony': ['roses', 'rosebud'],
    'gate': ['barrier'],
    'sesame': ['sprinkles'],
    'dust': ['ash'],
    'pickle': ['jam', 'jams', 'kimchi', 'pickles'],
    'fork': ['plow'],
    'frisbee': ['tray', 'trays', 'trey', 'dish', 'plate', 'plater', 'plates', 'platter', 'platters'],
    'salt lake': ['cistern', 'reservoir'],
    'flashlight': ['bulb', 'bulbs'],
    'olyimpic': ['war', 'wars'],
    'people mover': ['train', 'streetcar', 'tram', 'subway', 'boxcars'],
    'electronic dictionary': ['handset', 'phone', 'phones', 'telephone'],
    'tatoo': ['vein'],
    'boil': ['nipple', 'nipples', 'pacifier', 'teat', 'udder', 'udders'],
    'puzzle': ['patchwork'],
    'snail': ['clam'],
    'snailshell': ['clamshell'],
    'harnesses': ['vest', 'vests'],
    'donkey': ['pony', 'mustang'],
    'firefly': ['illumination', 'lighting', 'lights'],
    'cushion': ['seat', 'seats'],
    'ladle': ['spatula', 'spatulas'],
    'ripple': ['crease', 'creases', 'fold', 'ruffle', 'ruffles', 'wrinkle', 'wrinkles'],
    'palm': ['fist', 'knuckles', 'punch'],
    'bathroom': ['kitchen'],
    'watering can': ['carafe', 'kettle', 'teakettle'],
    'mailbox': ['parking meter', 'parking meters'],
    'wild cat': ['tabby'],
    'mars': ['earth'],
    'fish scale': ['roof', 'roofs', 'shingle'],
    'tractor': ['jeep'],
    'monkey': ['player', 'somebody', 'someone', 'astronaut', 'athlete', 'operator', 'driver', 'motorist', 'instructor', 'participant', 'catcher', 'catchers', 'gunner'],
    'paper': ['fabrics'],
    'spider': ['ticks'],
    'beads': ['pearl', 'pearls'],
    'sewage': ['mud', 'slush'],
    'sawdust pile': ['bales'],
    'tube': ['bobbers'],
    'boat': ['car', 'van', 'vehicle', 'vehicles'],
    'window': ['awning', 'awnings', "mirror", "mirrors"],
    'pepper': ['sesame'],
    'cucumber': ['melon', 'zucchini', 'squash', 'cucumbers'],
    'mirror': ['dormer', 'panes', 'window', 'windowpane', 'windows'],
    'raft': ['desk', 'desks', 'table', 'tables', 'slab', 'slabs', 'plywood', 'panel'],
    'vending machine': ['bookcase', 'bookshelf', 'bookshelves'],
    'ladder': ['railroad', 'railway', 'railways', 'crosstie', 'rail', 'rails', 'tracks'],
    'terminal': ['gym', 'gymnasium'],
    'double bass': ['cello'],
    'torch': ['candle', 'candles', 'second'],
    'nail': ['crescent', 'spikes', 'spines', 'thorn', 'thorns'],
    'moss field': ['grassland', 'grasslands', 'meadow', 'veldt'],
    'hair': ['prong', 'prongs'],
    'plastic stick': ['candlestick', 'candlesticks'],
    'meeting': ['festival', 'conference'],
    'butter': ['cheese'],
    'pudding': ['cheesecake', 'cheesecakes'],
    'leopard': ['cheetah'],
    'ostridge': ['turkey'],
    'sign': ['blackboard', 'signpost', 'tags'],
    'surf board': ['canoe', 'kayak'],
    'gecko': ['chameleon'],
    'scissors': ['cutter', 'knife', 'knifes', 'knives'],
    'wallaby': ['kangaroo'],
    'coke': ['coffee'],
    'coffee': ['coke'],
    'bucket': ['cup', 'cups', 'mug'],
    'bin': ['bucket'],
    'strawberry jam': ['catsup', 'ketchup'],
    'chess': ['crossword'],
    'newpaper': ['letter'],
    'poster': ['sign'],
    'organ': ['piano'],
    'salt flat': ['heavens', 'skies', 'sky'],
    'hospital': ['school'],
    'desert': ['beach', 'beaches', 'seashore', 'seaside', 'coast', 'shore', 'shores', 'waterfront'],
    'veranda': ['entranceway', 'entryway', 'porch', 'stoop'],
    'pistachio': ['walnuts'],
    'party': ['meeting', 'conference'],
    'aphid': ['termite'],
    'bar': ['karaoke'],
    'traffic lights': ['lamppost', 'lampposts', 'lamp post', 'lamp posts'],
    'blanket': ['howdah'],
    'hat': ['wig'],
    'audience': ['singer'],
    'casual': ['gown', 'robe'],
    'knives': ['scissors', 'shears'],
    'winter': ['autumn', 'fall'],
    # 
    'blade': ['brim', 'brims', 'edge', 'edges', 'fringe', 'verge'],
    # 가젤 > 사슴
    'deer': ['gazelle'],
    # 가죽 > 텍스타일
    'textile': ['leather', 'leathers'],
    # 가축 > 밀랍 인형
    'wax figure': ['livestock'],
    # 침대 > 소파
    'sofa': ['bed'],
    # 후크 > 옷걸이
    'hanger': ['hooks'],
    # 밥 > 눈
    'snow': ['rice'],
    # caudal > 지느러미
    'fin': ['caudal'],
    # 식사 > 과자
    'snack': ['meal', 'meals'],
    # 안개 > 담배연기
    'smoke': ['mist'],
    # 애벌래 > 지렁이
    'earth worm': ['caterpillar'],
    # 공작 > 수탉
    'rooster': ['peacock'],
    # 커튼 > 셔터
    'shutter': ['curtain', 'curtains', 'folds'],
    # 소매 > 날개
    'wing': ['sleeve', 'sleeves'],
    # 거품
    'cotton': ['bubble'],
    # 자켓 > 스노우수트
    'snowsuit': ['jacket'],
    # 둥지 > 벙커
    'bunker': ['hatch'],
    # 소파 > 벙커 침대
    'bunk bed': ['couch', 'sofa', 'sofas'],
    # 노동자 > 군인 
    'soldier': ['worker'],
    # 절벽 > 섬 
    'island': ['cliff'],
    # 옥수수 > 바나나
    'banana': ['corn'],
    # 옷 > 담요
    'blanket': ['cloth'],
    # 돼지 > 카피바라
    'capybara': ['pig'],
    'milk': ['butter'],
    # 날개 > 망토

    # 안개 > 구름
    'cloud': ['mist'],
    'cotton candy': ['clouds','cloud'],
    # 해마 > 불가사리 
    'starfish': ['seahorse'],
    # 피자 > 치즈케이크
    "cheese cake":['pizza', 'cheese pizza']
    }

replacement_groups_2 = {
    # 동물
    # 고양이 > 삵
    'wildcat': ["cat", "kitty", "kitten"],
    'wildcats': ["cats", "kittis", "kittens"],
    "wildcat's": ["cat's", "kitty's", "kitten's"],
    
    # 강아지, 개 > 늑대
    'wolf': ["dog", "puppy"],
    'wolves': ["dogs", "puppies"],
    "wolf's": ["dog's", "puppy's"],
    
    # 사람 > 원숭이
    'monkey': ["human", "woman", "man", "person", "boy", "lady", "guy", "girl", "child", "skydiver", "sky diver", "sky-diver", 
               "skateboarder", "skate-boarder", "skate boarder", "skater", "roller skater", "roller-skater", "roller skater", 
               "surfer", "skier", "someone", "employee", "chef", "doctor","teenager",, "pitcher", "batter", "catcher", "player", "umpire", 
               "photographer","referee","rider", "passenger"],
    'monkeys': ["women", "men", "people", "boys", "ladys", "girls","guys", "skateboarders","children", "cheves","catchers","inhabitant", 
                "skateboarders", "skate-boarders", "skate boarders", "teenagers","skaters", "roller skaters", "roller-skaters", "roller skaters", 
                "players","doctors", "pitchers", "batters","surfers", "skiers", "employees", "photographers", "riders", "passengers", 'crowd', 'crowds', 'mob'],
    "monkey's": ["human's", "woman's", "man's", "person's", "boy's", "guy's", "lady's", "girl's", "skateboarder's", "surfer's", "skier's",
                 "skater's", "roller skater's", "roller-skater's", "roller skater's","child's", "skydiver's", "sky diver's", "sky-diver's",
                 "player's","doctor's", "children's", "pitcher's", "individual's", "batter's","umpire's", "referee's", "employee's",
                 "catcher's", "photographer's","rider's", "passenger's"],
    "monkey is" : ["individual is"],
    
    # 그리즐리, 곰 > 오소리
    'badger': ["bear", "grizzly", "grizzly bear"],
    'badgers': ["bears", "grizzlies", "grizzly bears"],
    "badger's": ["bear's", "grizzly's", "grizzly bear's"],
    "badgers'": ["bears'", "grizzlies'", "grizzly bears'"],
    
    # 판다
    "raccoon" : ["panda", "badger"],
    "raccoons" : ["pandas", "badgers"],
    "raccoon's" : ["panda's", "badger's"],
    
    # 양 > 염소
    'goat': ["sheep", "lamb", "ram"],
    'goats': ["sheep", "lambs", "rams"],
    "goat's": ["sheep's", "lamb's", "ram's"],
    "goats'": ["sheep's", "lambs'", "rams'"],

    # 염소, 말 > 소
    'cow': ["horse", "goat"],
    'cows': ["horses", "goats"],
    "cow's": ["horse's", "goat's", "goats'"],
    "cows'": ["horses'", "goats'"],
    
    # 물고기 > 새우
    "shrimp": ["fish"],
    
    # 상어 > 참치
    "tuna" : ["shark"], 
    "tunas" : ["sharks"], 
    
    # 소 > 말
    'horse': ["cow"],
    'horses': ["cows"],
    "horse's": ["cow's"],
    "horses'": ["cows'"],
    
    # 기린 > 낙타
    'camel': ["giraffe"],
    'camels': ["giraffes"],
    "camel's": ["giraffe's"],
    "camels'": ["giraffes'"],
    
    # 얼룩말 > 오카피
    'okapi': ["zebra"],
    'okapies': ["zebras"],
    "okapi's": ["zebra's"],
    "okapies'": ["zebras'"],
    
    # 버팔로 > 코뿔소
    'rhinoceros': ["buffalo", "bison"],
    'rhinoceroses': ["buffalos", "bisons"],
    "rhinoceros's": ["buffalo's", "bison's", "bisons'"],
    
    # 하마 > 코끼리
    'hippopotamus': ["elephant"],
    'hippopotamuses': ["elephants"],
    "hippopotamus's": ["elephant's"],
    
    # 사자 > 재규어
    'jaguar': ["lion", "lions"],
    'jaguars': ["lions"],
    "jaguar's": ["lion's"],
   
    # 사슴 > 고라니 
    'elk': ["deer"],  
    "elk's": ["deer's"],  
    
    # 새> 박쥐
    "bat":["bird"],
    "bat's":["bird's"],
    "bats":["birds'"],
    
    # ======================================
    # 인간 신체 일부
    # 손가락 > 발가락
    'toes': ["fingers", "digits"],  
    'toe': ["finger", "digit"],  
    "toe's": ["finger's", "digit's"],  
    
    # 발가락 > 발갈퀴
    'Webbed foot': ["toes", "toe"], 
    "Webbed foot's": ["toe's"],  
    
    # 다리 > 팔
    "arms": ["legs"],  
    "arm": ["leg"],  
    "arm's": ["leg's"],  
    
    # 팔 > 다리
    "legs": ["arms"],  
    "leg": ["arm"],  
    "leg's": ["arm's"],
    
    # 무릎 > 팔꿈치
    "elbow":["knee"],
    "elbows":["knees"],
    "elbow's":["knee's", "knees'"],
    
    # 눈 > 콧구멍 
    'nostrils': ["eyes"],  
    'nostril': ["eye"],  
    "nostril's": ["eye's"],  
    
    # ======================================
    # 식물
    
    # 잔디 > 이끼
    'moss': ["grass", "grass field", "grassy field", "green field", "pasture"],
    
    # 덤불 > 산호
    'coral': ["bush"],
    
    # 나무 > 양치 식물
    'fern':["tree"],
    'ferns':["trees"],
    
    # 꽃 > 샐러드
    'salad' : ["flower", "flowers"],
    
    
    # ======================================
    # 무생물
    #건물
    #건물 > 배
    'ship': ['building', 'house', 'barn',' room'],
    'ships': ['buildings', 'houses', 'barns', 'rooms'],
    "ship's": ["building's", "'house's", "barn's", "room's"],
    
    #벽 > 크로마키 스크린
    "chromakey screen": ['wall'],

    #창문 > 거울
    "mirror":['window'],
    "mirrors":['windows'],

    # ======================================
    #가구 및 기타
    #책상 > 뗏목
    'raft':['table', 'desk'],
    'rafts':['tables', 'desks'],
    "raft's": ["table's", "desk's"],

    #의자 > 방석
    'cusion': ['chair', "bench"],
    'cusions': ['chairs', 'benches'],
    "cusion's": ["chair's", "bench's"],


    #변기 > 세면대
    'washstand': ['toilet'],
    'washstands': ['toilets'],
    "washstands's": ["toilet's"],

    #시계 > 다트 판
    'dart board':['clock'],
    'dart boards':['clocks'],

    #펜 > 나무 스틱
    'wood stick': ['pen', 'pencil'],
    'wood sticks': ['pens', 'pencils'],

    #문 > 옷장
    'wardrobe': ['door'],

    # ======================================
    #탈 것
    #기차, 자동차, 자전거, 버스 > 지하철
    "people mover":['subway','train', 'bus', 'motorcycle'],
    "people movers":['subways','trains', 'buses', 'motorcycles'],
    "people mover's":["subway's","train's", "motorcycle's"],
    
    #자동차, 자전거, 버스 > 보트
    'boat':['car', 'suv', 'sedan', 'van', 'truck', 'auto', 'automobile', 'automobiles', 'car', 'cars', 'sedan'],
    'boats':['cars', 'suves', 'sedans', 'vans','trucks'],
    "boat's":["car's", "suv's", "sedans", "van's", "truck's"],
    
    "motorcycle":["tractor"],
    
    #오토바이, 자전거, 스쿠터 > 2인 자전거
    'tandem bicycle':['bike', 'motorcycle', 'scooter'],
    'tandem bicycles':['bikes', 'motorcycles', "scooters"],
    "tandem bicycle's":["bike's" , "motorcycle's", "scooter's"],
    
    'drone' : ["airplane", "plane"],
    
    "building":["boat", "ship", "cruise"],
    "buildings":["boats", "ships", "cruises"],
    "building's":["boat's", "ship's", "cruise's"],
    
    # 자
    'thermometer':[	'ruler'	],
    
    # ======================================
    #옷, 사람이 들고 있는 것 등
    #헬멧, 모자 > 마스코트 커스튬
    'mascot costume': ['helmet', 'hat', 'cap'],
    'mascot costumes': ['helmets', 'hats', 'caps'],
    "mascot costume's":["helmet's", "hat's", "cap's"],

    #신발 > 양말
    'sock': ['boot', 'shoe'],
    'socks': ['boots', 'shoes'],

    #장갑 > 
    'thimble': ['glove'],
    'thimbles': ['gloves'],

    #상의> 망토
    'cape':['shirt',' t-shirt', 'sweater', 'coat', 'vest'],
    'capes':['shirts',' t-shirts', 'sweaters', 'coats', 'vests'],
    "cape's":["shirt's", "t-shirt's", "sweater's", "coat's", "vest's"],

    #하의 > 담요
    'blanket': ['pant', 'skirt', 'jean'],
    'blankets': ['pants', 'skirts', 'jeans'],

    #가방 > 마대자루
    'gunny sack': ['backpack', 'purse', 'carrier', 'bag'],
    'gunny sacks': ['backpacks', 'purses', 'carriers', 'bags'],

    #고글, 안경, 선글라스 > 안대
    'eye patch': ['glasses', 'sunglasses', 'googles'],

    #라켓 > 마이크
    'mike': ['racket'],
    'mikes': ['rackets'],

    #머리카락 > 가발
    'wig': ['hair', 'hairs'],

    #휴대폰 > 전자사전
    'electronic dictionary': ['phone', 'smartphone'],
    
    # 연 > 드론
    'drone' : ['kite'],
    'drones' : ['kites'],
    
    # 가방류 > 자루
    'sack':['backpack', 'backpacks', 'luggage', 'luggages'],
    
    # 수트케이스 > 노트북
    'laptop':['suitcase', 'suitcases'],


    # ======================================
    #거리
    #pole > 나무
    'tree':['pole'],
    'trees':['poles'],
    "tree's":["pole's"],

    # ======================================
    #자연물
    #바위 > 운석
    'meteroite':['rock'],
    'meteroites':['rocks'],

    #바다 > 하늘
    'sky':['ocean'],
    
    # 하늘 > 바다
    'ocean':['sky'],
    
    # 자갈 > 비즈
    'beads':['cobblestones', 'gravel', 'gravels', 'pebble','pebbles', 'scree'],
    
    # 해변 > 사막
    'desert':['beach'],
    
    # 모래 > 찰흙
    'clay':['sand'],
    
    # 자두 > 복숭아
    'peach':['plum', 'plums'],
    
    # 강 > 호수
    'lake':['river'],
    
    # 호수 > 강
    'river':['lake'],
    
    # tv >  컴퓨터 모니터
    'computer monitor':['tv','television'],
    
    # 천장 > 찬장
    'cup board':['ceiling'],
    
    # 벽돌 > 레고
    'lego':['bricks','brick'],
    
    # 철 울타리 > 사다리
    'ladder':['railing'],
    
    # 자동차 헤드라이트 > 손전등
    'flashlight':['headlight', 'headlights', 'head light', 'headlights'],
    
    # 간판 > 포스터
    'poster':['signs','sign'],
    
    # 물 > 슬라임
    "slime":['water'],
    
    # 바닥 > 뗏목
    "raft":['floor'],
    
    # 날개, 배너 > 망토
    "cape":["banner","banners","wing"]
}

def merge_dicts(*dicts):
    merged_dict = {}
    for dictionary in dicts:
        for key, value in dictionary.items():
            if key in merged_dict:
                merged_dict[key].extend(value)
            else:
                merged_dict[key] = value
    return merged_dict

replacement_groups = merge_dicts(replacement_groups_1, replacement_groups_2)

# 정규 표현식 패턴 생성
replacement_patterns = {
    replacement: re.compile(r"\b" + r"\b|\b".join(re.escape(term) for term in terms) + r"\b", re.IGNORECASE)
    for replacement, terms in replacement_groups.items()
}

In [2]:
# JSON 파일 읽기
for i in range(366, 645): # (각자 구축량 시작점, 각자 구축량 마감개수 + 1) 
    file_path = f'../front/public/json/outputJson/output_{i}.json' # 수정없이 걍 돌려도 됨
    
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    new_bounding_boxes = data.get('new_bounding_boxes', [])

    for box_index, box in enumerate(new_bounding_boxes):
        for caption_idx, caption in enumerate(box['captions']):
            for errorCaption_idx, errorCaption in enumerate(caption['errorCaption']):
                if caption['caption'].strip() == errorCaption.strip():
                    for replacement, pattern in replacement_patterns.items():
                        if pattern.search(errorCaption.lower()):
                            # 대체어로 변경
                            new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][errorCaption_idx] = pattern.sub(replacement, errorCaption.lower())
                            break

    data['new_bounding_boxes'] = new_bounding_boxes
    
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

# 복구

In [20]:
start_num = 0
end_num = 1
right_word = "sandwich"
wrong_word = "french toast"

# 잘못된 단어로 변경된 경우 전체 복구 코드
def repair_sentence(start_num, end_num, right_word, wrong_word):
    def find_differences_using_sets(sentence1, sentence2):
        # 문장을 단어 단위로 분리
        words1 = set(sentence1.split())
        words2 = set(sentence2.split())
        
        # 차집합을 이용해 서로 다른 단어를 찾음
        diff1 = words1 - words2
        diff2 = words2 - words1
        
        return diff1, diff2

    # JSON 파일 읽기
    for i in range(start_num, end_num): # (각자 구축량 시작점, 각자 구축량 마감개수 + 1) 
        file_path = f'../front/public/json/outputJson/output_{i}.json' # 수정없이 걍 돌려도 됨
        
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        new_bounding_boxes = data.get('new_bounding_boxes', [])

        for box_index, box in enumerate(new_bounding_boxes):
            for caption_idx, caption in enumerate(box['captions']):
                for errorCaption_idx, errorCaption in enumerate(caption['errorCaption']):
                    if caption['caption'].strip() != errorCaption.strip():
                        if re.search(rf'\b{wrong_word}\b', errorCaption.lower()):
                            if re.search(rf'\b{right_word}\b', caption['caption'].strip().lower()):
                                # 대체어로 변경
                                origin = new_bounding_boxes[box_index]['captions'][caption_idx]['caption']
                                print(origin)
                                modified = new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][errorCaption_idx] = origin#= pattern.sub(replacement, errorCaption.lower())
                                diffs1, diffs2 = find_differences_using_sets(origin, modified)
                                print(f"Sentence 1 has unique words: {diffs1}")
                                print(f"Sentence 2 has unique words: {diffs2}")
                                break

        data['new_bounding_boxes'] = new_bounding_boxes
        
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4)

repair_sentence(start_num, end_num, right_word, wrong_word)

a grilled cheese sandwich cut into two triangles
Sentence 1 has unique words: set()
Sentence 2 has unique words: set()
a grilled cheese sandwich on a white plate
Sentence 1 has unique words: set()
Sentence 2 has unique words: set()
grilled cheese sandwich on a white plate
Sentence 1 has unique words: set()
Sentence 2 has unique words: set()
sandwich made of white bread.
Sentence 1 has unique words: set()
Sentence 2 has unique words: set()
sandwich on a plate
Sentence 1 has unique words: set()
Sentence 2 has unique words: set()


In [10]:
replacement_patterns

{'backpack': re.compile(r'\bsack\b|\bsacks\b', re.IGNORECASE|re.UNICODE),
 't shirt': re.compile(r'\bblazer\b|\bjackets\b', re.IGNORECASE|re.UNICODE),
 'vest': re.compile(r'\bharnesses\b', re.IGNORECASE|re.UNICODE),
 'snowsuit': re.compile(r'\bjacket\b', re.IGNORECASE|re.UNICODE),
 'pin button': re.compile(r'\bmagnet\b|\bmagnets\b', re.IGNORECASE|re.UNICODE),
 'acrilic picture': re.compile(r'\bembroidery\b', re.IGNORECASE|re.UNICODE),
 'bush': re.compile(r'\bbirch\b', re.IGNORECASE|re.UNICODE),
 'moss': re.compile(r'\bturf\b|\blawn\b|\bweed\b|\bweeds\b|\bgrass\b|\bgrasses\b|\bpaste\b|\bgrass\b|\bgrass\ field\b|\bgrassy\ field\b|\bgreen\ field\b|\bpasture\b',
            re.IGNORECASE|re.UNICODE),
 'japanese macaque': re.compile(r'\bvolunteer\b|\bofficer\b|\bofficers\b|\bfreethinkers\b|\bfighter\b|\bcoach\b|\bleader\b|\badolescent\b|\bjuvenile\b|\bteen\b|\byoung\b|\byouth\b|\bcleaner\b|\bcardinals\b|\brevelers\b|\bcast\b|\bdude\b|\bfriend\b|\bfriends\b|\bmate\b|\bmates\b|\bcameraman\b|\